In [1]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='a034b98b2f6f49ee8757523c7f2ed350' name='default'>

In [2]:
from aiida_kkr.calculations import KkrimpCalculation
from masci_tools.io.parsers.kkrimp_parser_functions import KkrimpParserFunctions

In [3]:
critical_files = [
            ('outfile', KkrimpCalculation._DEFAULT_OUTPUT_FILE),
            ('out_log', KkrimpCalculation._OUTPUT_000),
            ('out_pot', KkrimpCalculation._OUT_POTENTIAL),
            ('out_timing', KkrimpCalculation._OUT_TIMING_000),
            ('out_enersp_at', KkrimpCalculation._OUT_ENERGYSP_PER_ATOM),
            ('out_enertot_at', KkrimpCalculation._OUT_ENERGYTOT_PER_ATOM),
        ]

In [4]:
ret = orm.load_node(43648).outputs.retrieved
for name in critical_files:
    print(name)
    with ret.open(name[1], 'rb') as _f:
        with open('test_files/'+name[1], 'wb') as _f2:
            _f2.writelines(_f.readlines())

('outfile', 'out_kkrimp')
('out_log', 'out_log.000.txt')
('out_pot', 'out_potential')
('out_timing', 'out_timing.000.txt')
('out_enersp_at', 'out_energysp_per_atom_eV')
('out_enertot_at', 'out_energytotal_per_atom_eV')


In [32]:
named_file_handles = {key: 'test_files/'+val for key, val in critical_files}
# named_file_handles['out_log'] = 'test_files/out_log.000.txt_small'
named_file_handles

{'outfile': 'test_files/out_kkrimp',
 'out_log': 'test_files/out_log.000.txt',
 'out_pot': 'test_files/out_potential',
 'out_timing': 'test_files/out_timing.000.txt',
 'out_enersp_at': 'test_files/out_energysp_per_atom_eV',
 'out_enertot_at': 'test_files/out_energytotal_per_atom_eV'}

In [33]:
# now we can parse the output files
success, msg_list, out_dict = KkrimpParserFunctions().parse_kkrimp_outputfile(
    {}, named_file_handles, debug=False, timings=True
)

start parser
parser 1 0.22146868705749512
parser 2 5.29449987411499
parser 3 0.34971070289611816
parser 3 2.881960391998291
parser 3 0.0004031658172607422
parser 3 1.430511474609375e-05
parser 3 0.0074803829193115234
parser 3 1.2492520809173584
parser 3 0.009205818176269531
parser 3 0.0889441967010498
parser 3 0.0037970542907714844
parser 3 0.049860477447509766
parser 3 0.12708425521850586
parser 3 0.004278659820556641
parser 3 0.14441394805908203
parser total 10.433269500732422


In [12]:
from masci_tools.io.parsers.kkrparser_functions import get_rms, parse_array_float

def test():
    rms_charge, rms_spin, result_atoms_last_charge, result_atoms_last_spin = get_rms(named_file_handles['outfile'],
                                                                                     named_file_handles['out_log'],
                                                                                     debug=False)
    
%timeit test()

1.39 s ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
debug = False
def test():
    rms_charge = parse_array_float(named_file_handles['outfile'], 'average rms-error', [2, '=', 1, 0], ['D', 'E'], debug=debug)

%timeit test()

297 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:

def test():
    rms_charge_atoms = parse_array_float(named_file_handles['out_log'], 'rms-error for atom', [2, '=', 1, 0], ['D', 'E'])

%timeit test()

408 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from masci_tools.io.parsers.kkrparser_functions import get_outfile_txt, search_string

In [34]:
outfile = named_file_handles['outfile']

%timeit tmptxt = get_outfile_txt(outfile)

1.7 ms ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
tmptxt = get_outfile_txt(outfile)

In [39]:
searchstring = 'average rms-error'
replacepair = ['D', 'E']
splitinfo = [2, '=', 1, 0]
import numpy as np

def test():
    itmp = 0
    res = []
    # print(len(tmptxt))
    while itmp >= 0:
        itmp = search_string(searchstring, tmptxt)
        if debug:
            print(('in parse_array_float (itmp, searchstring, outfile):', itmp, searchstring, outfile))
        if itmp >= 0:
            tmpval = tmptxt.pop(itmp)
            if replacepair is not None:
                tmpval = tmpval.replace(replacepair[0], replacepair[1])
            if splitinfo[0] == 1:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]])
            elif splitinfo[0] == 2:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
            else:
                raise ValueError('splitinfo[0] has to be either 1 or 2')
            res.append(tmpval)
    res = np.array(res)
    return res
    
%timeit res = test()

1.97 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
def search(tmptxt, searchstring, replacepair, splitinfo):
    itmp = 0
    res = []
    while itmp >= 0:
        itmp = search_string(searchstring, tmptxt)
        if debug:
            print(('in parse_array_float (itmp, searchstring, outfile):', itmp, searchstring, outfile))
        if itmp >= 0:
            tmpval = tmptxt.pop(itmp)
            if replacepair is not None:
                tmpval = tmpval.replace(replacepair[0], replacepair[1])
            if splitinfo[0] == 1:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]])
            elif splitinfo[0] == 2:
                tmpval = float(tmpval.split(splitinfo[1])[splitinfo[2]].split()[splitinfo[3]])
            else:
                raise ValueError('splitinfo[0] has to be either 1 or 2')
            res.append(tmpval)
    res = np.array(res)
    return res

def test():
    tmptxt = get_outfile_txt(outfile)
    res = search(tmptxt, searchstring, replacepair, splitinfo)
    return res
    
%timeit res = test()

293 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:

tmptxt = get_outfile_txt(outfile)
def test():
    res = search(tmptxt, searchstring, replacepair, splitinfo)
    return res
    
%timeit res = test()

1.92 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
